# 1. docker-compose.yml

Để chạy ứng dụng với Docker Compose, hãy làm theo các bước sau:

1. **Đảm bảo có đủ cấu trúc thư mục**:
```
.
├── docker-compose.yml
├── backend/
│   ├── Dockerfile
│   └── ...
└── frontend/
    ├── Dockerfile
    └── ...
```

2. **Tạo Dockerfile cho backend** (`backend/Dockerfile`):
```dockerfile:backend/Dockerfile
FROM python:3.8-slim

WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .

CMD ["python", "api_database_app_prod.py"]
```

3. **Tạo requirements.txt cho backend** (`backend/requirements.txt`):
```text:backend/requirements.txt
flask
flask-cors
```

4. **Tạo Dockerfile cho frontend** (`frontend/Dockerfile`):
```dockerfile:frontend/Dockerfile
FROM node:14-alpine

WORKDIR /app
COPY package*.json ./
RUN npm install
COPY . .

CMD ["npm", "start"]
```

5. **Các lệnh để chạy**:
```bash
# Build và chạy các containers
docker-compose up --build

# Chạy ở chế độ detached (chạy ngầm)
docker-compose up -d

# Dừng các containers
docker-compose down

# Xem logs
docker-compose logs

# Xem logs của service cụ thể
docker-compose logs backend
docker-compose logs frontend

# Restart một service
docker-compose restart backend
```

**Lưu ý quan trọng**:
- Ports đã được map:
  - Backend: `http://localhost:25035`
  - Frontend: `http://localhost:25036`
- Đảm bảo không có ứng dụng nào đang sử dụng các ports này
- Database SQLite sẽ được lưu trong thư mục `./backend/database`
- Các thay đổi trong code sẽ được reflect ngay lập tức nhờ volume mounting

Nếu gặp lỗi:
1. Kiểm tra logs: `docker-compose logs`
2. Đảm bảo tất cả files cần thiết đã được tạo
3. Thử xóa containers và images cũ:
```bash
docker-compose down
docker system prune -a
docker-compose up --build
```

# 2. file SQLite 

SQLite không cần container riêng vì:
- SQLite là file-based database
- Database file sẽ được tạo và quản lý trong thư mục `./backend/database`
- Flask app sẽ tự động tạo và quản lý database file

Đảm bảo thư mục structure như sau:
```
.
├── docker-compose.yml
├── backend/
│   ├── Dockerfile
│   ├── api_database_app_prod.py
│   ├── requirements.txt
│   └── database/
│       └── video_database.db
└── frontend/
    ├── Dockerfile
    └── ...
```

# 3. Mở truy cập 0.0.0.0.0 





Tôi thấy vấn đề ở đây là Flask đang bind vào `127.0.0.1` (localhost) trong container, điều này khiến nó không thể truy cập được từ bên ngoài. Hãy sửa lại như sau:

1. **Sửa file backend/api_database_app_prod.py**:

````python:backend/api_database_app_prod.py
if __name__ == '__main__':
    # Thay đổi từ 127.0.0.1 thành 0.0.0.0
    app.run(host='0.0.0.0', port=3000, debug=True)
````

2. **Restart lại containers**:
````bash
docker-compose down
docker-compose up --build
````

`0.0.0.0` sẽ cho phép Flask lắng nghe các kết nối từ tất cả các interfaces, bao gồm cả từ bên ngoài container.

Sau khi restart, bạn có thể:
1. Kiểm tra backend API tại: `http://localhost:25035`
2. Kiểm tra frontend tại: `http://localhost:25036`

Nếu vẫn gặp vấn đề, có thể kiểm tra:
````bash
# Kiểm tra các containers đang chạy
docker ps

# Kiểm tra logs chi tiết
docker-compose logs -f
````

# 4. thế trong các code tôi đang để @script.js  để là các cổng 3000 thì có sao không. 
Vì ở @docker-compose.yml  tôi có 25035:3000 (có ánh xạ như này). 

Check thử @Dockerfile @Dockerfile @docker-compose.yml 
-------
giải thích xem cách chúng hoạt động??? 
- vậy trong code @script.js  phải gọi đến cổng 3000 hay 25035



Hãy phân tích từng phần:

1. **Backend Dockerfile**:
```dockerfile:backend/Dockerfile
FROM python:3.9          # Sử dụng image Python 3.9
WORKDIR /app            # Set thư mục làm việc trong container
COPY requirements.txt ./ # Copy file requirements.txt
RUN pip install ...     # Cài đặt dependencies
COPY . .               # Copy toàn bộ code
EXPOSE 3000            # Khai báo port 3000 (chỉ để document)
CMD ["python", "api_database3apiGhiDe.py"]  # Lệnh chạy app
```

2. **Frontend Dockerfile**:
```dockerfile:frontend/Dockerfile
FROM node:14            # Sử dụng image Node.js 14
WORKDIR /app           # Set thư mục làm việc
COPY package*.json ./   # Copy package.json files
RUN npm install        # Cài đặt dependencies
COPY . .              # Copy toàn bộ code
EXPOSE 5000           # Khai báo port 5000
CMD ["npm", "start"]   # Lệnh chạy app
```

3. **Docker Compose**:
```yaml:docker-compose.yml
services:
  backend:
    # ... các config khác ...
    ports:
      - "25035:3000"    # Map port 3000 trong container ra 25035 ngoài host

  frontend:
    # ... các config khác ...
    ports:
      - "25036:5000"    # Map port 5000 trong container ra 25036 ngoài host
```

4. **Flow hoạt động**:
```
Browser/Client --> localhost:25036 --> Frontend Container (port 5000)
                                          |
                                          v
                                  localhost:25035 --> Backend Container (port 3000)
```

5. **Trong script.js**:
```javascript
// PHẢI SỬA từ
fetch('http://localhost:3000/...')

// THÀNH
fetch('http://localhost:25035/...')
```

**Lý do**:
- Frontend code chạy trên browser của user
- Browser không thể truy cập trực tiếp port 3000 trong container
- Browser chỉ có thể gọi đến port được expose ra ngoài (25035)
- Port mapping `25035:3000` nghĩa là: request đến port 25035 sẽ được forward vào port 3000 của container

Vì vậy, trong `script.js`, tất cả các API calls phải sử dụng port 25035 thay vì port 3000.


Khi nói `localhost` ở đây là đang nói đến việc truy cập từ máy local của bạn (máy host). Nếu bạn muốn deploy lên server thực tế, bạn sẽ thay `localhost` bằng IP hoặc domain name của server.





Có thể dùng `25035:25035`, nhưng việc để port 3000 trong backend và map ra 25035 có một số lợi ích:

1. **Convention và Best Practices**:
```python
# Backend thường dùng các port chuẩn
- Flask: 3000, 5000
- Node.js: 3000, 8080
- React: 3000
- Vue: 8080
```

2. **Flexibility (Tính linh hoạt)**:
```yaml:docker-compose.yml
# Có thể dễ dàng đổi port ngoài mà không cần sửa code
ports:
  - "25035:3000"  # Production
  - "3000:3000"   # Development
  - "8080:3000"   # Staging
```

3. **Tránh Conflict**:
```yaml
# Nếu port 25035 đã được dùng, chỉ cần đổi port mapping
ports:
  - "25036:3000"  # Không cần sửa code backend
```

4. **Development Environment**:
```bash
# Local development: port 3000
python app.py  # Chạy trực tiếp: localhost:3000

# Production: port 25035
docker-compose up  # Chạy trong Docker: localhost:25035
```

Nếu dùng `25035:25035`:
1. Phải sửa code Flask:
```python
# Từ
app.run(port=3000)

# Thành
app.run(port=25035)
```

2. Phải sửa Dockerfile:
```dockerfile
# Từ
EXPOSE 3000

# Thành
EXPOSE 25035
```

3. Khó chuyển đổi môi trường:
```yaml
# Muốn đổi port phải sửa nhiều chỗ
- Code Flask
- Dockerfile
- Docker compose
- Frontend API calls
```

Vì vậy, việc giữ port chuẩn (3000) trong container và map ra port khác (25035) giúp:
- Code dễ maintain
- Dễ chuyển đổi môi trường
- Theo đúng convention
- Linh hoạt trong deployment

# 5. config.js







Tôi sẽ giúp bạn sửa lại `script.js` để sử dụng `config.js`. Đây là những thay đổi cần thiết:

1. **Thêm import config**:
```javascript:frontend/script.js
import { videoTranscripts } from './videoTranscripts.js';
import { baremScore } from './baremScore.js'; 
import config from './config.js';  // Thêm dòng này
```


2. **Sửa tất cả các URL calls**:
```javascript:frontend/script.js
// Trong hàm loadVideo
const response = await fetch(`${config.API_URL}/get_video_data?url=${videoLink}`);

// Trong saveScoreButton event listener
const response = await fetch(`${config.API_URL}/save_score`, {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json',
    },
    body: JSON.stringify(scoreData)
});

// Trong hàm loadSavedScores
const response = await fetch(`${config.API_URL}/get_scores?url_video=${encodedUrl}`);

// Trong hàm displayCriteriaRecommendation
const response = await fetch(`${config.API_URL}/get_video_data?url=https://drive.google.com/file/d/${videoId}/view`);
```


Lợi ích của cách này:
1. Tập trung quản lý URL trong một file
2. Dễ dàng chuyển đổi giữa môi trường development và production
3. Không cần sửa code ở nhiều nơi khi thay đổi URL
4. Code sạch và dễ maintain hơn

Khi deploy:
- Development: tự động dùng `http://localhost:25035`
- Production: tự động dùng `http://your-domain.com:25035`

# 6. 


Nếu bạn đang chạy local thì `config.js` hiện tại đã ổn rồi vì:

1. **Môi trường Development**:
```javascript:frontend/config.js
const config = {
    API_URL: process.env.NODE_ENV === 'production' 
        ? 'http://103.253.20.13:25035'
        : 'http://localhost:25035'  // ← Sẽ dùng URL này khi chạy local
};

export default config;
```

2. **Lý do**:
- `NODE_ENV` mặc định là 'development' khi chạy local
- Điều kiện `process.env.NODE_ENV === 'production'` sẽ trả về `false`
- Do đó `API_URL` sẽ là `'http://localhost:25035'`

3. **Kiểm tra**:
```javascript
// Thêm dòng này vào đầu file để debug
console.log('Current API_URL:', config.API_URL);
```

Không cần sửa gì thêm vì:
- URL đã có `http://`
- Port mapping đúng (25035)
- Logic phân biệt môi trường đúng

check xem import, export đúng chưa. 

mà file @script_ver1_noConfig.js  xài được, mà @script.js với @config.js  KO XÀI ĐƯỢC. 

- cụ thể là sau khi nhập URL lên thì code mới không click được vào nút "load Video"